## Import Packages

In [56]:
import geopandas as gpd
import hvplot.pandas
import cartopy.crs as ccrs
import panel as pn

In [57]:
def check_and_reproject(df):
    """
    Checks the coordinate reference system (CRS) of the input DataFrame and reprojects it to EPSG:4326 if necessary.

    Parameters:
    df (GeoDataFrame): The input DataFrame containing spatial data.

    Returns:
    GeoDataFrame: The reprojected DataFrame.
    """
    if df.crs != 'EPSG:4326':
        df = df.to_crs('EPSG:4326')
    else:
        df=df
    return df

In [58]:
def process_paddocks(land_parcels, user_paddocks):
    """
    Args:
        land_parcels (geopandas.GeoDataFrame): GeoDataFrame representing the land parcels.
        user_paddocks (geopandas.GeoDataFrame): GeoDataFrame representing the user-defined paddocks.

    Returns:
        tuple: A tuple containing the following:
            - land_parcels_unify (geopandas.GeoDataFrame): GeoDataFrame representing the dissolved land parcels.
            - land_parcels_centroid (geopandas.GeoSeries): GeoSeries representing the centroid of the land parcels.
            - inside_areas (geopandas.GeoDataFrame): GeoDataFrame representing the areas of the user-input boundaries that fall inside the land parcels.
            - outside_areas (geopandas.GeoDataFrame): GeoDataFrame representing the areas areas of the user-input boundaries that outside the land parcels.
    """
    land_parcels_unify = land_parcels.dissolve()
    land_parcels_centroid = land_parcels_unify.centroid

    difference_areas = gpd.overlay(user_paddocks, land_parcels, how='symmetric_difference') 
    outside_areas = gpd.overlay(difference_areas, land_parcels, how='difference')
    inside_areas = user_paddocks.clip(land_parcels)
    
    return land_parcels_unify, land_parcels_centroid, inside_areas, outside_areas


In [59]:
def export_geojson(inside_areas, outside_areas):
    """
    Export two files:
     - the modified user-input boundaries
     - the areas that fall outside the land parcels
     
    The files will be projected to EPSG:4326 before being exported.
    """
    output_crs = '4326'

    inside_areas.to_file('output-data/modified-user-boundaries.geojson', driver='GeoJSON', crs=output_crs)
    outside_areas.to_file('output-data/outside-parcel-user-boundaries.geojson', driver='GeoJSON', crs=output_crs)


In [60]:
# TODO: add widgets to let user drop in their onw files
cpes = gpd.read_file('input-data/CPES/cpes.shp')
paddocks = gpd.read_file('input-data/Paddock boundaries/paddock-boundaries.shp').to_crs(cpes.crs)

# set up map tiles for basemap
map_tiles = hv.element.tiles.EsriImagery().opts(width=600, height=400)

cpes_reproject=check_and_reproject(cpes)
paddocks_reproject=check_and_reproject(paddocks)

In [61]:
land_parcels_unify, land_parcels_centroid, inside_areas, outside_areas = process_paddocks(cpes_reproject, paddocks_reproject)

/tmp/ipykernel_2664/1636014326.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  land_parcels_centroid = land_parcels_unify.centroid


In [62]:
parcels_map = land_parcels_unify.hvplot(alpha=0.5, geo=True)
outside_map = outside_areas.hvplot(geo=True).opts(title='submitted areas that fall outside land parcels')
inside_map = inside_areas.hvplot(geo=True).opts(title='submitted areas that are inside land parcels')

In [63]:
pn.Row(
    pn.Column(
    (map_tiles * parcels_map * outside_map),
    (map_tiles * parcels_map * inside_map)
    )
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'df93b7a9-eb14-4a54-8157-85fc16473afc': {'version…

In [64]:
export_geojson(inside_areas, outside_areas)